In [1]:
%pip install pyvisa
%pip install pyvisa-py


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyvisa as visa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
rm = visa.ResourceManager('@py')
print(rm.list_resources())


('ASRL1::INSTR', 'TCPIP::172.18.26.21::INSTR', 'TCPIP::172.18.26.24::INSTR', 'TCPIP::172.18.26.30::INSTR', 'TCPIP::172.18.26.31::INSTR', 'TCPIP::172.18.26.32::INSTR', 'TCPIP::172.18.26.38::INSTR', 'TCPIP::172.18.26.39::INSTR', 'TCPIP::172.18.26.40::INSTR', 'TCPIP::172.18.26.50::INSTR', 'TCPIP::172.18.26.55::INSTR', 'TCPIP::172.18.26.56::INSTR', 'TCPIP::172.18.26.58::INSTR', 'TCPIP::172.18.26.62::INSTR', 'TCPIP::172.18.26.64::INSTR', 'TCPIP::172.18.26.71::INSTR', 'TCPIP::172.18.26.88::INSTR', 'TCPIP::172.18.26.89::INSTR', 'TCPIP::172.18.26.91::INSTR', 'TCPIP::172.18.26.92::INSTR', 'TCPIP::172.18.26.98::INSTR', 'TCPIP::172.18.26.100::INSTR', 'TCPIP::172.18.26.104::INSTR', 'TCPIP::172.18.26.105::INSTR', 'TCPIP::172.18.26.106::INSTR', 'TCPIP::172.18.26.113::INSTR', 'TCPIP::172.18.26.115::INSTR', 'TCPIP::172.18.26.116::INSTR', 'TCPIP::172.18.26.118::INSTR', 'TCPIP::172.18.26.121::INSTR', 'TCPIP::172.18.26.134::INSTR', 'TCPIP::172.18.26.150::INSTR', 'TCPIP::172.18.26.154::INSTR', 'TCPIP::172

In [ ]:
# 请将以下地址替换为你设备的实际VISA地址
dm3068_address = 'USB0::0x1AB1::0x0C94::DM3O141200002::INSTR' 
dg4202_address = 'USB0::0x1AB1::0x0641::DG4F191600000::INSTR' # Example address, replace with your DG4202 address

try:
    dm3068 = rm.open_resource(dm3068_address)
    dg4202 = rm.open_resource(dg4202_address)
    print("DM3068 and DG4202 connected successfully.")
    
    # Optionally query identification for verification
    print(f"DM3068 ID: {dm3068.query('*IDN?')}")
    print(f"DG4202 ID: {dg4202.query('*IDN?')}")

except Exception as e:
    print(f"Error connecting to instruments: {e}")
    # Exit or handle error appropriately
    exit()

# Reset and clear any previous settings
dm3068.write('*RST')
dg4202.write('*RST')

# Configure DM3068 for DC Voltage measurement
dm3068.write(':MEASure:VOLTage:DC?') # Set to DC voltage measurement, query to get current value


In [ ]:
start_voltage = -0.2
step_voltage = 0.004
num_points = 101

# Configure DG4202 to output DC voltage
dg4202.write(':SOURce1:FUNCtion DC')
dg4202.write(':OUTPut1 ON') # Enable output


In [ ]:
vin_values = []
vout_values = []

for i in range(num_points):
    vin = start_voltage + i * step_voltage
    vin_values.append(vin)
    
    # Set DG4202 output voltage
    dg4202.write(f':SOURce1:VOLTage:LEVel:IMMediate:AMPLitude {vin}')
    
    # Measure voltage with DM3068
    # Adding a small delay might be necessary for the voltage to settle
    # time.sleep(0.1) 
    vout = float(dm3068.query(':MEASure:VOLTage:DC?'))
    vout_values.append(vout)
    
    print(f"Setting Vin: {vin:.4f}V, Measured Vout: {vout:.4f}V")


In [ ]:
df = pd.DataFrame({'Vin': vin_values, 'Vout': vout_values})
df.to_csv('measurement_data.csv', index=False)
print("Data saved to measurement_data.csv")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(vin_values, vout_values, 'o-', label='Measured Vout')
plt.plot(vin_values, vin_values, '--', label='Ideal Vout (Vin)') # Ideal line for comparison
plt.xlabel('Input Voltage (Vin)')
plt.ylabel('Output Voltage (Vout)')
plt.title('Vin vs Vout Characteristics')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Convert to numpy arrays for easier calculation
vin_np = np.array(vin_values)
vout_np = np.array(vout_values)

# --- 1. Gain, Offset, and Relative Nonlinearity (基于最佳拟合线) ---
gain, offset_V = np.polyfit(vin_np, vout_np, 1)
ideal_vout_bestfit = gain * vin_np + offset_V
max_deviation_V = np.max(np.abs(vout_np - ideal_vout_bestfit))

output_range = np.max(vout_np) - np.min(vout_np)
nonlinearity_percent = (max_deviation_V / output_range) * 100 if output_range > 1e-9 else 0

print(f"Calculated Gain: {gain:.4f}")
print(f"Calculated Offset: {offset_V:.4f}V")
print(f"Nonlinearity (%): {nonlinearity_percent:.4f}%")

# Calculate INL (Integral Non-Linearity)
# INL = Vout_measured - Vout_ideal_fitted (using best fit line)

inl = vout_np - ideal_vout_bestfit

# Calculate DNL (Differential Non-Linearity)
# DNL is the deviation of actual step size from ideal step size
# Ideal step size is `step_voltage * gain` (based on best fit gain)

dnl = np.diff(vout_np) - (step_voltage * gain)


plt.figure(figsize=(10, 6))
plt.plot(vin_np, inl, 'o-')
plt.xlabel('Input Voltage (Vin)')
plt.ylabel('INL (V)')
plt.title('Integral Non-Linearity (INL) based on Best-Fit Line')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(vin_np[1:], dnl, 'o-') # DNL has one less point than Vin
plt.xlabel('Input Voltage (Vin)')
plt.ylabel('DNL (V)')
plt.title('Differential Non-Linearity (DNL) based on Best-Fit Line')
plt.grid(True)
plt.show()

print(f"Max INL: {np.max(np.abs(inl)):.4f}V")
print(f"Min INL: {np.min(inl):.4f}V")
print(f"Max DNL: {np.max(np.abs(dnl)):.4f}V")
print(f"Min DNL: {np.min(dnl):.4f}V")

# Plotting Vin vs Vout with Best-Fit Line
plt.figure(figsize=(10, 6))
plt.plot(vin_np, vout_np, 'o-', label='Measured Vout')
plt.plot(vin_np, ideal_vout_bestfit, '--', label='Best-Fit Line')
plt.xlabel('Input Voltage (Vin)')
plt.ylabel('Output Voltage (Vout)')
plt.title('Vin vs Vout Characteristics with Best-Fit Line')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Disable output and close connections
dg4202.write(':OUTPut1 OFF')
dm3068.close()
dg4202.close()
rm.close()
print("Instruments disconnected.")
